<a href="https://colab.research.google.com/github/hongqin/AI4Health/blob/main/kaggle_synthetic_healthcare_explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# the data set is
# https://www.kaggle.com/code/skuracha/eda-on-healthcare-dataset

# Objective
# Identify key factors affecting the Test Results

import kagglehub
import os
import kagglehub
import os

# Download the latest version of the dataset
path = kagglehub.dataset_download("prasad22/healthcare-dataset")

print("Path to dataset files:", path)

# Initialize variable for file path
file_path = None

# Find the single file in the directory
for root, dirs, files in os.walk(path):
    if files:  # If there are files in the current directory
        file_path = os.path.join(root, files[0])  # Assume only one file is present
        break  # Stop after finding the first file

if file_path:
    print("Full path to the file:", file_path)
else:
    print("No files found in the dataset directory.")


Path to dataset files: /root/.cache/kagglehub/datasets/prasad22/healthcare-dataset/versions/2
Full path to the file: /root/.cache/kagglehub/datasets/prasad22/healthcare-dataset/versions/2/healthcare_dataset.csv


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [7]:
data = pd.read_csv(file_path)

data.head()

Name  Age  Gender Blood Type Medical Condition Date of Admission  \
0  Bobby JacksOn   30    Male         B-            Cancer        2024-01-31   
1   LesLie TErRy   62    Male         A+           Obesity        2019-08-20   
2    DaNnY sMitH   76  Female         A-           Obesity        2022-09-22   
3   andrEw waTtS   28  Female         O+          Diabetes        2020-11-18   
4  adrIENNE bEll   43  Female        AB+            Cancer        2022-09-19   

             Doctor                    Hospital Insurance Provider  \
0     Matthew Smith             Sons and Miller         Blue Cross   
1   Samantha Davies                     Kim Inc           Medicare   
2  Tiffany Mitchell                    Cook PLC              Aetna   
3       Kevin Wells  Hernandez Rogers and Vang,           Medicare   
4    Kathleen Hanna                 White-White              Aetna   

   Billing Amount  Room Number Admission Type Discharge Date   Medication  \
0    18856.281306          328         Urgent     2024-02-02  Paracetamol   
1    33643.327287          265      Emergency     2019-08-26    Ibuprofen   
2    27955.096079          205      Emergency     2022-10-07      Aspirin   
3    37909.782410          450       Elective     2020-12-18    Ibuprofen   
4    14238.317814          458         Urgent     2022-10-09   Penicillin   

   Test Results  
0        Normal  
1  Inconclusive  
2        Normal  
3      Abnormal  
4      Abnormal

In [8]:
data.shape

(55500, 15)

In [9]:
# Columns in data
data.columns

Index(['Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition',
       'Date of Admission', 'Doctor', 'Hospital', 'Insurance Provider',
       'Billing Amount', 'Room Number', 'Admission Type', 'Discharge Date',
       'Medication', 'Test Results'],
      dtype='object')

In [10]:
# Info About columns
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                55500 non-null  object 
 1   Age                 55500 non-null  int64  
 2   Gender              55500 non-null  object 
 3   Blood Type          55500 non-null  object 
 4   Medical Condition   55500 non-null  object 
 5   Date of Admission   55500 non-null  object 
 6   Doctor              55500 non-null  object 
 7   Hospital            55500 non-null  object 
 8   Insurance Provider  55500 non-null  object 
 9   Billing Amount      55500 non-null  float64
 10  Room Number         55500 non-null  int64  
 11  Admission Type      55500 non-null  object 
 12  Discharge Date      55500 non-null  object 
 13  Medication          55500 non-null  object 
 14  Test Results        55500 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 6.4

In [11]:
# Checking duplicate values
data.loc[data.duplicated()]

Name  Age  Gender Blood Type Medical Condition  \
50023        Samuel joYCe   56    Male         O+         Arthritis   
50040    KIMBErly vasqueZ   26    Male         A-           Obesity   
50055      SCoTT tHOrNton   55  Female         B-           Obesity   
50070         wiLliam LEe   27  Female         O+         Arthritis   
50078  chRistOPheR NoRrIs   38    Male        AB-         Arthritis   
...                   ...  ...     ...        ...               ...   
55461      connOR coMPTon   63    Male         A+            Asthma   
55462       alYSsA mIlLER   35  Female         A-          Diabetes   
55464        ChRIs huGHeS   35  Female        AB-           Obesity   
55484     keNNEtH alvarez   80    Male         O+            Cancer   
55491       lInDA grIFfin   47  Female         O+           Obesity   

      Date of Admission             Doctor                    Hospital  \
50023        2022-11-03     Krista Hartman              Thomas-Ballard   
50040        2023-10-23   Jennifer Bennett                   Cowan Inc   
50055        2021-04-08      Joshua Ingram                  Keller LLC   
50070        2020-01-31  Lauren Montgomery               Winters-Simon   
50078        2023-01-14       Denise Blair                 Inc Morales   
...                 ...                ...                         ...   
55461        2021-08-21     Jonathan Allen  and Willis Mullins, Bowers   
55462        2022-06-30         Ryan Price           Shelton-Gallagher   
55464        2024-02-28      Katelyn Perry                Lyons-Hansen   
55484        2022-05-05      Andrew Conner               Sons Mayo and   
55491        2020-06-21     Danielle Davis    and King Jordan Johnson,   

      Insurance Provider  Billing Amount  Room Number Admission Type  \
50023   UnitedHealthcare    15654.830587          157         Urgent   
50040   UnitedHealthcare    38142.109678          313         Urgent   
50055           Medicare    30894.904511          455         Urgent   
50070              Cigna    46229.434901          153       Elective   
50078           Medicare     9072.127142          452      Emergency   
...                  ...             ...          ...            ...   
55461           Medicare     1936.702824          375      Emergency   
55462   UnitedHealthcare     2210.460898          289       Elective   
55464         Blue Cross    11889.154513          128      Emergency   
55484              Cigna    45653.802310          114       Elective   
55491              Aetna    15715.551079          310      Emergency   

      Discharge Date   Medication  Test Results  
50023     2022-11-22      Aspirin  Inconclusive  
50040     2023-11-18   Penicillin      Abnormal  
50055     2021-04-15      Aspirin      Abnormal  
50070     2020-02-13      Lipitor        Normal  
50078     2023-02-12   Penicillin        Normal  
...              ...          ...           ...  
55461     2021-09-16  Paracetamol        Normal  
55462     2022-07-27   Penicillin        Normal  
55464     2024-03-14  Paracetamol      Abnormal  
55484     2022-05-17      Aspirin        Normal  
55491     2020-06-25      Lipitor      Abnormal  

[534 rows x 15 columns]

In [13]:
data['Date of Admission'] = pd.to_datetime(data['Date of Admission'])
data['Discharge Date'] = pd.to_datetime(data['Discharge Date'])

In [14]:
data['Gender'].value_counts()

Gender
Male      27774
Female    27726
Name: count, dtype: int64

In [15]:
data['Blood Type'].value_counts()

Blood Type
A-     6969
A+     6956
AB+    6947
AB-    6945
B+     6945
B-     6944
O+     6917
O-     6877
Name: count, dtype: int64

In [17]:
df = data.drop(columns = ['Name','Doctor','Hospital', 'Room Number'])
df.info()
df['Days_Admitted'] = (df['Discharge Date'] - df['Date of Admission']).dt.days + 1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Age                 55500 non-null  int64         
 1   Gender              55500 non-null  object        
 2   Blood Type          55500 non-null  object        
 3   Medical Condition   55500 non-null  object        
 4   Date of Admission   55500 non-null  datetime64[ns]
 5   Insurance Provider  55500 non-null  object        
 6   Billing Amount      55500 non-null  float64       
 7   Admission Type      55500 non-null  object        
 8   Discharge Date      55500 non-null  datetime64[ns]
 9   Medication          55500 non-null  object        
 10  Test Results        55500 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 4.7+ MB
